In [1]:

import sounddevice as sd
import numpy as np
import matplotlib.pyplot as plt
from tkinter import *
from tkinter import ttk
import time
from copy import copy
import mido


from synt.function import *
from synt.const import *
from synt.synt import *
from synt.osc import *

from synt.envolv import *
from synt.effects import *
from synt.instrumento import *
from synt.mixer import *

import os

In [2]:
'''MOSTAR ONDAS COPIANDOLAS PARA QUE NO SE ALTERE EL FRAME'''
from copy import copy
def showOsc(osc, time, chunks=False):
    _osc = copy(osc)
    signal = np.zeros(0)
    _chunks = int(time*(SRATE+CHUNK)/CHUNK)
    if chunks:
        _chunks = time
    for i in range(_chunks):
        signal = np.concatenate((signal, _osc.next()))
    plt.plot(signal)

In [3]:
'''REPRODUCTOR'''
input = None

def callback(outdata, frames, time, status):
    global input
    # print('entro')
    if input is not None:
        bloque = input.next()
        # convertimos formato (CHUNK,) a (CHUNK,1) para que adecuarlo a sounddevice
        outdata[:] = bloque.reshape(-1, 1)
    else:
        # si no hay datos, reproducimos silencio
        outdata[:] = np.zeros((CHUNK, 1))


# stream de salida con callBack
stream = sd.OutputStream(samplerate=SRATE, channels=2, callback=callback, blocksize=CHUNK)
stream.start()

In [4]:
raise Exception("Para de ejecutar!!") #(comentar para ejecutar todo de una)

Exception: Para de ejecutar!!

In [11]:
root = Tk()
tk = LabelFrame(root, text="Prueba", bg="#808090")

text = Text(tk,height=4,width=40)
text.pack(side=BOTTOM)

tk.pack(side=LEFT)

slider=Scale(tk, from_=10, to=100, resolution=1., orient=HORIZONTAL, label="silder", length=100)
slider.set(55)
slider.pack()

root.mainloop()


In [6]:

root = Tk()
tk = LabelFrame(root, text="Prueba", bg="#808090")

# c = C(440, nombre="Const", tk=tk, show=True)
# c.doShow()
amp = Sine(C(30, show=True), 
           min=C(0, show=True), 
           max=C(1, show=True), 
           
           show=True)

phase=Sine(C(880, show=True), 
           max=C(1, show=True),
           min=C(0, show=True),
           show=True 
           ) 

s = Sine(
    C(440, show=True), 
    amp=amp,
    phase=phase,
    nombre="Seno", 
    show=True)
# s.doShow(tk)

synt = Synt(freq=C(550), onda=s, amp=C(1), show=True, nombre="Synt")

synt.doShow(tk)
tk.pack(side=LEFT)
# root.pack(side=TOP)
input = synt
root.mainloop()
input = None


In [5]:

root = Tk()
tk = LabelFrame(root, text="Prueba", bg="#808090")

# c = C(440, nombre="Const", tk=tk, show=True)
# c.doShow()
amp = Sine(C(30, show=True), 
           min=C(0), 
           max=C(1, show=True), 
           show=True)

phase=Sine(C(880, show=True), 
           max=C(1, show=True),
           min=C(0, show=True),
           show=True 
           ) 

s = Sine(
    C(440, show=True), 
    amp=amp,
    phase=phase,
    nombre="Seno", 
    show=True
    )
# s.doShow(tk)

tk.pack(side=LEFT)

a = C(1, show=True, nombre="general")
# a.doShow(tk)

synt = Synt(C(1), s, amp=a, show=True, nombre="Synt")
env = EnvInstrumento(.9, .1, .7, .9, show=True)

inst = Instrumento(synt, env) # cutre
inst.doShow(tk)
# root.pack(side=TOP)
# input = inst
input=inst
root.mainloop()
input = None


ALSA lib pcm.c:8772:(snd_pcm_recover) underrun occurred


noteOn 48
523.2511306011972
noteOff 48
off
noteOn 50
587.3295358348151
noteOff 50
noteOn 52
659.2551138257398
noteOff 52
off
off
noteOn 48
261.6255653005986
noteOff 48
noteOn 50
293.6647679174076
noteOff 50
noteOn 52
329.6275569128699
noteOff 52
off
off
off
noteOn 48
261.6255653005986
noteOff 48
noteOn 48
261.6255653005986
noteOff 48
noteOn 48
261.6255653005986
noteOff 48
noteOn 48
261.6255653005986
noteOff 48
off
noteOn 48
523.2511306011972
noteOff 48
noteOn 48
523.2511306011972
noteOff 48
noteOn 48
523.2511306011972
noteOff 48
noteOn 48
523.2511306011972
noteOff 48
noteOn 48
523.2511306011972
noteOff 48
noteOn 48
523.2511306011972
noteOff 48
noteOn 48
523.2511306011972
noteOff 48
noteOn 48
523.2511306011972
noteOff 48
noteOn 48
523.2511306011972
noteOff 48
noteOn 48
523.2511306011972
noteOff 48
noteOn 48
523.2511306011972
noteOff 48
noteOn 48
523.2511306011972
noteOff 48
noteOn 48
523.2511306011972
noteOff 48
noteOn 48
523.2511306011972
noteOff 48
noteOn 48
523.2511306011972
noteOff 